In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import pandas as pd
import numpy as np
import shutil
import os

Using TensorFlow backend.


In [2]:
DATA_DIR = 'Data'
TRAIN_DIR = 'train'
VAL_DIR = 'val'
trainingData = pd.read_csv(os.path.join(DATA_DIR, 'ISIC-2017_Training_Part3_GroundTruth.csv'))
x = []
labels = trainingData.melanoma.loc[:400]
'''
for i, img in enumerate(trainingData.image_id.unique()[:400]):
    inPath = os.path.join(DATA_DIR, 'ISIC-2017_Training_Data', img + '.jpg')
    isMel = '1' if labels[i] == 1 else '0'
    outpath = os.path.join(DATA_DIR, TRAIN_DIR if i < 300 else VAL_DIR, isMel,img + '.jpg')
    shutil.copy(inPath,outpath)
'''

"\nfor i, img in enumerate(trainingData.image_id.unique()[:400]):\n    inPath = os.path.join(DATA_DIR, 'ISIC-2017_Training_Data', img + '.jpg')\n    isMel = '1' if labels[i] == 1 else '0'\n    outpath = os.path.join(DATA_DIR, TRAIN_DIR if i < 300 else VAL_DIR, isMel,img + '.jpg')\n    shutil.copy(inPath,outpath)\n"

In [3]:
train_y = labels[:300]
val_y = labels[300:]

In [4]:
batch_size = 16
img_width = 299
img_height = 299
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rotation_range=95,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range = 0.2,
        height_shift_range = 0.2)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        os.path.join(DATA_DIR, TRAIN_DIR),
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        os.path.join(DATA_DIR, VAL_DIR),
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

Found 300 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [5]:
model = applications.xception.Xception(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

In [7]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss = "binary_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [15]:
'''
for layer in model.layers[:5]:
    layer.trainable = False
'''
import keras
from sklearn.metrics import roc_auc_score

class auc_roc_history(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.aucs = []
        self.losses = []
            
    def on_train_end(self, logs={}):
        return
 
    def on_epoch_begin(self, epoch, logs={}):
        print self.model.inputs
        return
 
    def on_epoch_end(self, epoch, logs={}):
        print self.model.inputs, self.model.targets
        self.losses.append(logs.get('loss'))
        y_pred = self.model.predict(self.model.inputs, verbose = 0)
        auc_score = roc_auc_score(self.model.targets, y_pred)
        self.aucs.append(auc_score)
        logging.info("interval evaluation - epoch: {:d} - score: {:.6f}".format(epoch, score))
        return
 
    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        return

In [16]:
epochs = 5
nb_train_samples = 300
nb_validation_samples = 100
history = auc_roc_history()
model_final.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size, 
        verbose = 1,
        callbacks = [history])

[<tf.Tensor 'input_1:0' shape=(?, 299, 299, 3) dtype=float32>]
Epoch 1/5


FailedPreconditionError: Attempting to use uninitialized value training/SGD/Variable_1
	 [[Node: training/SGD/Variable_1/read = Identity[T=DT_FLOAT, _class=["loc:@training/SGD/Variable_1"], _device="/job:localhost/replica:0/task:0/cpu:0"](training/SGD/Variable_1)]]

Caused by op u'training/SGD/Variable_1/read', defined at:
  File "/Users/shashankbhushan/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/shashankbhushan/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/shashankbhushan/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOL

In [19]:
model_final.test_data

AttributeError: 'Model' object has no attribute 'test_data'